## waste management

In [1]:
pip install opencv-python


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 3.4 MB/s eta 0:00:12
   - -------------------------------------- 1.3/39.5 MB 3.9 MB/s eta 0:00:10
   -- ------------------------------------- 2.4/39.5 MB 4.5 MB/s eta 0:00:09
   --- ------------------------------------ 3.4/39.5 MB 4.6 MB/s eta 0:00:08
   ---- ----------------------------------- 4.7/39.5 MB 4.8 MB/s eta 0:00:08
   ----- ---------------------------------- 5.8/39.5 MB 5.0 MB/s eta 0:00:07
   ------- -------------------------------- 7.3/39.5 MB 5.3 MB/s eta 0:00:07
   -------- ------------------------------- 8.7/39.5 MB 5.4 MB/s eta 0:00:06
   --------- ------------------------------ 9.7/39.5 MB 5.4 MB/s eta 0:00:06
   ----------- ---------------------------- 11.3/39.5 MB 5.5 MB/s eta 0:00:06
   ------------ --------------------------- 12.1/39.5 MB 5.4 MB/s eta 0:00:06
   ------------- -------------------------- 13.4/39.5 MB 5.4 MB/s eta 0:00:05
  

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarning('ignore')
 
train_path = "dataset/TRAIN" 
test_path = "dataset/TEST"
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import plot_model
from glob import glob
 
#visualization
x_data = []
y_data = []
for category in glob(train_path+"/*"):
    for file in tqdm(glob(category+"/*")):
        img_array = cv2.imread(file)
        img_array = cv2.cvtColor(img_array,cv2.COLOR_BGR2RGB)
        x_data.append(img_array)
        y_data.append(category.split('/')[-1])
data = pd.DataFrame({'image':x_data,"label":y_data})
 
data.shape()
 
colors = ["#a0d157", "#c48bb8"]
plt.pie(data.label.value_counts(), labels = ["Organic","Recyclable"], autopct = '%0.2f%%', colors = colors, startangle = 90, explode = [0.05, 0.05] ) #explode is optional
plt.show()

ModuleNotFoundError: No module named 'pandas'